<a href="https://colab.research.google.com/github/Yvonne511/donor-analysis-react-d3/blob/main/graph_data_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [164]:
df_bounce = pd.read_csv('drive/MyDrive/unhcr/Updated Data/data with sentid/bounces.csv')
df_click = pd.read_csv('drive/MyDrive/unhcr/Updated Data/data with sentid/clicks.csv')
df_open = pd.read_csv('drive/MyDrive/unhcr/Updated Data/data with sentid/opens.csv')
df_sent = pd.read_csv('drive/MyDrive/unhcr/Updated Data/data with sentid/sent.csv')
df_donate = pd.read_csv('drive/MyDrive/unhcr/Provided By UNHCR/data/updated_lgadv_contacts.csv')

In [165]:
df_sent.head()

,contactid,eventdate,eventtype,sendid
0,0031K00003B42UhQAJ,4/28/22 8:02,Sent,891711
1,0031K00003B42UhQAJ,4/19/22 8:04,Sent,889440
2,0031K00003B42fvQAB,4/28/22 8:02,Sent,891711
3,0031K00003B438DQAR,5/17/22 15:01,Sent,906349
4,0031K00003B44irQAB,5/8/22 16:01,Sent,902017


In [166]:
df_open.head()

,contactid,eventdate,eventtype,sendid
0,0031K0000386BslQAE,4/19/22 7:37,Open,883709
1,0031K0000386BslQAE,4/19/22 7:37,Open,877618
2,0031K00003AMz7nQAD,4/19/22 7:41,Open,886508
3,0031K00003AO0VcQAL,4/19/22 8:04,Open,889440
4,0031K00003877mQQAQ,4/19/22 8:05,Open,889440


In [167]:
df_click.head()

,contactid,eventdate,eventtype,sendid,urlid,url
0,0031K00003B2PQmQAN,4/21/2022 12:08:19 AM,Click,765495,365099,https://www.unrefugees.org/news/army-veteran-a...
1,0031K00003B2PQmQAN,4/21/2022 12:10:08 AM,Click,765495,337899,https://www.unrefugees.org/news/not-even-monso...
2,0031K00003AQPzjQAH,4/20/2022 10:41:07 AM,Click,765495,365099,https://www.unrefugees.org/news/army-veteran-a...
3,0031K00003BNevBQAT,4/21/2022 11:46:52 AM,Click,890270,1094332,https://www.unrefugees.org/news/learn-how-you-...
4,0031K00003BOdezQAD,4/21/2022 11:26:15 AM,Click,890270,158534,http://www.facebook.com/UNREFUGEES?utm_medium=...


In [168]:
df_bounce.head()

,contactid,eventdate,eventtype,sendid,bouncecategory,bouncereason
0,0031K000039asp1QAA,4/19/2022 9:08:12 AM,Bounce,889450,Soft bounce,Inactive Account
1,0031K000039YaEtQAK,5/17/2022 3:07:32 PM,Bounce,906349,Soft bounce,Mailbox Full
2,0031K00003869ArQAI,3/31/2022 11:42:56 AM,Bounce,878820,Soft bounce,Mailbox Full
3,0031K0000387jNqQAI,4/13/2022 9:07:52 AM,Bounce,886509,Soft bounce,Mailbox Full
4,0031K0000387ooZQAQ,4/13/2022 8:11:56 AM,Bounce,886508,Soft bounce,Mailbox Full


In [169]:
## Sorted df_sent by sendid
df_sent.dtypes
df_sent['eventdate'] = pd.to_datetime(df_sent.eventdate)
df_sent.head()
df_sent.dtypes
df_sent.sort_values(by = ['eventdate'])

,contactid,eventdate,eventtype,sendid
383041,0031K0000385TdQQAU,2022-01-01 10:03:00,Sent,765490
435941,0031K0000385ToKQAU,2022-01-01 10:03:00,Sent,765490
375255,0031K0000385TqaQAE,2022-01-01 10:03:00,Sent,765490
319044,0031K0000385TQbQAM,2022-01-01 10:03:00,Sent,765490
33353,0031K0000385TrdQAE,2022-01-01 10:03:00,Sent,765490
...,...,...,...,...
277520,0031K00003AMyXLQA1,2022-09-14 11:01:00,Sent,964823
393921,0031K00003AMyXLQA1,2022-09-15 10:01:00,Sent,965533
36503,0031K000039ZRHzQAO,2022-09-15 10:01:00,Sent,965533
165302,0031K00003Bc6pQQAR,2022-09-15 10:03:00,Sent,890310


# df_sent Cleanup

In [170]:
## Users from each sendid
df_sent['sendid'].value_counts()

948099    14235
944011    12979
944781    12887
937762    12359
933990    12040
          ...  
927007        1
927008        1
953850        1
846210        1
847699        1
Name: sendid, Length: 319, dtype: int64

In [171]:
df_sent = df_sent.drop_duplicates(subset=['contactid'], keep='first')

In [172]:
##df_sent.to_csv('send_updated.csv')
##files.download('send_updated.csv')

In [173]:
df_sent['contactid'].value_counts()

0031K00003B42UhQAJ    1
0031K00003C2zhuQAB    1
0031K00003C3FpzQAF    1
0031K00003BbPCgQAN    1
0031K00003BbPBxQAN    1
                     ..
0031K00003875aNQAQ    1
0031K00003875PtQAI    1
0031K0000386onxQAA    1
0031K0000386ol8QAA    1
0031K000039bpvpQAA    1
Name: contactid, Length: 19232, dtype: int64

# df_open Cleanup

In [174]:
df_sent['sendid'].value_counts()

765495    850
918835    827
765490    796
870311    673
765498    604
         ... 
889501      1
779337      1
890505      1
948217      1
949003      1
Name: sendid, Length: 170, dtype: int64

In [175]:
df_sent['sendid'].value_counts().sum()

19232

In [176]:
first_sendid = df_sent['sendid'].drop_duplicates().tolist()
print(len(first_sendid))

170


In [177]:
df_open = df_open[df_open['sendid'].isin(first_sendid)]
df_open['eventdate'] = pd.to_datetime(df_open.eventdate)
df_open.sort_values(by = ['eventdate'])
df_open['contactid'].value_counts()

0031K000039Zl4HQAS    184
0031K0000385sadQAA    172
0031K0000386JrQQAU    155
0031K00003Bz6Q3QAJ    145
0031K00003AMfPwQAL    142
                     ... 
0031K00003B2r60QAB      1
0031K0000387T5GQAU      1
0031K000038623tQAA      1
0031K00003C1TiRQAV      1
0031K00003BdLCfQAN      1
Name: contactid, Length: 10244, dtype: int64

In [178]:
##df_open.to_csv('open_updated.csv')
##files.download('open_updated.csv')

# df_click Cleanup

In [179]:
df_click['eventdate'] = pd.to_datetime(df_click.eventdate)
df_click.sort_values(by = ['eventdate'])
df_click = df_click[df_click['sendid'].isin(first_sendid)]
df_click['contactid'].value_counts()

0031K000039YlFmQAK    359
0031K00003AN54FQAT    305
0031K00003AMvcfQAD    242
0031K00003Bz6Q3QAJ    212
0031K00003C1En7QAF    147
                     ... 
0031K000039Z5pHQAS      1
0031K0000385aEKQAY      1
0031K0000385sRqQAI      1
0031K00003C19wnQAB      1
0031K00003C0NHpQAN      1
Name: contactid, Length: 2694, dtype: int64

In [180]:
##df_click.to_csv('click_updated.csv')
##files.download('click_updated.csv')

# df_bounce Cleanup

In [181]:
df_bounce['eventdate'] = pd.to_datetime(df_bounce.eventdate)
df_bounce.sort_values(by = ['eventdate'])
df_bounce = df_bounce[df_bounce['sendid'].isin(first_sendid)]
df_bounce['contactid'].value_counts()

0031K0000385XxEQAU    14
0031K00003879wdQAA    12
0031K0000386QLaQAM    11
0031K0000387ooZQAQ     8
0031K00003872mgQAA     8
                      ..
0031K0000386EK3QAM     1
0031K000039bB9lQAE     1
0031K000039bNGJQA2     1
0031K00003APg5eQAD     1
0031K00003C1AlYQAV     1
Name: contactid, Length: 288, dtype: int64

In [182]:
##df_bounce.to_csv('bounce_updated.csv')
##files.download('bounce_updated.csv')

# df_donate Cleanup

In [183]:
df_donate
df_donate['createddate'] = pd.to_datetime(df_donate.createddate)
df_donate['first_gift_date__c'] = pd.to_datetime(df_donate.first_gift_date__c)
df_donate.sort_values(by = ['first_gift_date__c'])

,contactid,campaignid,name,leadgen,advocacy,accountid,createddate,membership_type__c,first_gift_date__c,first_gift_amount__c,mrc_date__c,mrc_amount__c,current_calendar_year_total_amount__c,current_calendar_year_total_transactions__c,previous_calendar_year_total_amount__c,previous_calendar_year_total_transaction__c,total_amount__c,total_transactions__c,email,phone
276,0031K0000385mZqQAI,7011K000001HSNYQA4,2021_YEMEN_FB,1,0,0011K00002THf9rQAD,2022-01-06 22:02:59+00:00,Individual Donor,2022-01-06,30.0,2022-03-06,30.0,90.0,3.0,0.0,0.0,90.0,3.0,1,1
477,0031K0000385uFxQAI,7011K000001HSNYQA4,2021_YEMEN_FB,1,0,0011K00002THmQXQA1,2022-01-10 03:49:57+00:00,Individual Donor,2022-01-10,10.0,2022-08-10,10.0,80.0,8.0,0.0,0.0,80.0,8.0,1,1
248,0031K0000385j8nQAA,7011K000001HSNYQA4,2021_YEMEN_FB,1,0,0011K00002THbyIQAT,2022-01-06 12:10:02+00:00,Individual Donor,2022-01-23,20.0,2022-01-23,20.0,20.0,1.0,0.0,0.0,20.0,1.0,1,1
1817,0031K0000386naMQAQ,7011K000001HtcmQAC,2021_ClimateChange_Web,0,1,0011K00002UNkHUQA1,2022-01-22 16:48:54+00:00,Individual Donor,2022-01-28,60.0,2022-01-28,60.0,60.0,1.0,0.0,0.0,60.0,1.0,1,1
1183,0031K00003871mfQAA,7011K000001HonxQAC,2021_Afg_YT,1,0,0011K00002UNxO2QAL,2022-01-25 23:25:51+00:00,Mid-Level,2022-01-31,50.0,2022-09-09,20.0,1280.0,16.0,0.0,0.0,1280.0,16.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24708,0031K00003C2IN8QAN,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,0,1,0011K00002Ww8r0QAB,2022-06-14 14:44:49+00:00,Donor Prospect,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
24709,0031K00003C2INGQA3,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,0,1,0011K00002Ww8r8QAB,2022-06-14 14:44:49+00:00,Donor Prospect,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
24710,0031K00003C2INHQA3,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,0,1,0011K00002Ww8r9QAB,2022-06-14 14:44:49+00:00,Donor Prospect,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
24711,0031K00003C2INIQA3,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,0,1,0011K00002Ww8rAQAR,2022-06-14 14:44:49+00:00,Donor Prospect,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [184]:
unique_contactid = df_sent['contactid'].tolist()
print(len(unique_contactid))

19232


In [185]:
df_donate = df_donate[["contactid", "campaignid", "name", "first_gift_date__c", "first_gift_amount__c", "membership_type__c", "email", "phone"]]
## df_donate = df_donate.drop_duplicates(subset=['contactid'], keep='first')
df_donate = df_donate[df_donate['contactid'].isin(unique_contactid)]
df_donate

,contactid,campaignid,name,first_gift_date__c,first_gift_amount__c,membership_type__c,email,phone
0,0031K0000385TNIQA2,7011K000001HSNYQA4,2021_YEMEN_FB,NaT,NaN,Donor Prospect,1,1
1,0031K0000385TZJQA2,7011K000001HSNYQA4,2021_YEMEN_FB,NaT,NaN,Donor Prospect,1,0
3,0031K0000385Tc8QAE,7011K000001HSNYQA4,2021_YEMEN_FB,NaT,NaN,Donor Prospect,1,1
4,0031K0000385TdQQAU,7011K000001HSNYQA4,2021_YEMEN_FB,NaT,NaN,Donor Prospect,1,0
5,0031K0000385TnlQAE,7011K000001HSNYQA4,2021_YEMEN_FB,NaT,NaN,Donor Prospect,1,1
...,...,...,...,...,...,...,...,...
24707,0031K00003C2IN7QAN,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,NaT,NaN,Donor Prospect,1,0
24708,0031K00003C2IN8QAN,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,NaT,NaN,Donor Prospect,1,0
24709,0031K00003C2INGQA3,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,NaT,NaN,Donor Prospect,1,0
24711,0031K00003C2INIQA3,7011K00000231T9QAI,2022_RefugeesEverywhere_Adv_DailyKos,NaT,NaN,Donor Prospect,1,0


In [186]:
df_donated = df_donate[df_donate['membership_type__c'] != 'Donor Prospect']
df_donated['eventdate'] = df_donated['first_gift_date__c']
df_donated['donatemount'] = df_donated['first_gift_amount__c']
df_donated['eventtype'] = 'Donate'
df_donated.drop(['first_gift_date__c', 'email', 'phone', 'membership_type__c', 'campaignid', 'name', 'first_gift_amount__c'], axis=1, inplace=True)
df_donated

,contactid,eventdate,donatemount,eventtype
31,0031K0000385UdIQAU,2022-02-26,25.0,Donate
248,0031K0000385j8nQAA,2022-01-23,20.0,Donate
276,0031K0000385mZqQAI,2022-01-06,30.0,Donate
333,0031K0000385qrwQAA,2022-05-10,7.0,Donate
346,0031K0000385rAlQAI,2022-02-27,60.0,Donate
...,...,...,...,...
24460,0031K00003BbhQEQAZ,NaT,NaN,Donate
24461,0031K00003Bbjz7QAB,NaT,NaN,Donate
24484,0031K00003C2dFnQAJ,2022-06-22,20.6,Donate
24485,0031K00003C2dJpQAJ,2022-06-24,30.0,Donate


In [187]:
df_donated['sendid'] = df_sent['sendid']
df_donated['sendid'] = df_donated['contactid'].map(df_sent.set_index('contactid')['sendid']).fillna('')

In [188]:
df_donated

,contactid,eventdate,donatemount,eventtype,sendid
31,0031K0000385UdIQAU,2022-02-26,25.0,Donate,866012
248,0031K0000385j8nQAA,2022-01-23,20.0,Donate,882582
276,0031K0000385mZqQAI,2022-01-06,30.0,Donate,878819
333,0031K0000385qrwQAA,2022-05-10,7.0,Donate,877617
346,0031K0000385rAlQAI,2022-02-27,60.0,Donate,886507
...,...,...,...,...,...
24460,0031K00003BbhQEQAZ,NaT,NaN,Donate,927013
24461,0031K00003Bbjz7QAB,NaT,NaN,Donate,924164
24484,0031K00003C2dFnQAJ,2022-06-22,20.6,Donate,957534
24485,0031K00003C2dJpQAJ,2022-06-24,30.0,Donate,927012


# Combined

In [189]:
df_sent

,contactid,eventdate,eventtype,sendid
0,0031K00003B42UhQAJ,2022-04-28 08:02:00,Sent,891711
2,0031K00003B42fvQAB,2022-04-28 08:02:00,Sent,891711
3,0031K00003B438DQAR,2022-05-17 15:01:00,Sent,906349
4,0031K00003B44irQAB,2022-05-08 16:01:00,Sent,902017
7,0031K00003B44nrQAB,2022-04-27 10:10:00,Sent,765498
...,...,...,...,...
325633,0031K00003CjvJNQAZ,2022-07-08 10:07:00,Sent,765496
335313,0031K000039c0bkQAA,2022-08-12 11:16:00,Sent,949003
371402,0031K000039abgZQAQ,2022-03-10 09:02:00,Sent,867410
371645,0031K000039bBqPQAU,2022-03-10 09:02:00,Sent,867410


In [190]:
df_open

,contactid,eventdate,eventtype,sendid
0,0031K0000386BslQAE,2022-04-19 07:37:00,Open,883709
1,0031K0000386BslQAE,2022-04-19 07:37:00,Open,877618
2,0031K00003AMz7nQAD,2022-04-19 07:41:00,Open,886508
3,0031K00003AO0VcQAL,2022-04-19 08:04:00,Open,889440
4,0031K00003877mQQAQ,2022-04-19 08:05:00,Open,889440
...,...,...,...,...
137839,0031K0000387ooUQAQ,2022-02-11 02:30:00,Open,765495
137841,0031K0000387ootQAA,2022-02-11 19:16:00,Open,765495
137842,0031K0000386QEAQA2,2022-02-13 03:51:00,Open,847679
137843,0031K0000386oRTQAY,2022-02-13 15:23:00,Open,765490


In [191]:
df_bounce

,contactid,eventdate,eventtype,sendid,bouncecategory,bouncereason
0,0031K000039asp1QAA,2022-04-19 09:08:12,Bounce,889450,Soft bounce,Inactive Account
1,0031K000039YaEtQAK,2022-05-17 15:07:32,Bounce,906349,Soft bounce,Mailbox Full
2,0031K00003869ArQAI,2022-03-31 11:42:56,Bounce,878820,Soft bounce,Mailbox Full
3,0031K0000387jNqQAI,2022-04-13 09:07:52,Bounce,886509,Soft bounce,Mailbox Full
4,0031K0000387ooZQAQ,2022-04-13 08:11:56,Bounce,886508,Soft bounce,Mailbox Full
...,...,...,...,...,...,...
793,0031K00003BdOy1QAF,2022-09-11 10:06:59,Bounce,962694,Technical/Other bounce,Network Error
794,0031K00003BcsZCQAZ,2022-09-08 10:13:09,Bounce,962694,Soft bounce,Mailbox Full
795,0031K00003BcqhFQAR,2022-08-30 10:11:53,Bounce,957561,Soft bounce,Mailbox Full
796,0031K00003C1AlYQAV,2022-08-30 09:11:18,Bounce,957560,Soft bounce,Mailbox Full


In [192]:
df_click

,contactid,eventdate,eventtype,sendid,urlid,url
0,0031K00003B2PQmQAN,2022-04-21 00:08:19,Click,765495,365099,https://www.unrefugees.org/news/army-veteran-a...
1,0031K00003B2PQmQAN,2022-04-21 00:10:08,Click,765495,337899,https://www.unrefugees.org/news/not-even-monso...
2,0031K00003AQPzjQAH,2022-04-20 10:41:07,Click,765495,365099,https://www.unrefugees.org/news/army-veteran-a...
3,0031K00003BNevBQAT,2022-04-21 11:46:52,Click,890270,1094332,https://www.unrefugees.org/news/learn-how-you-...
4,0031K00003BOdezQAD,2022-04-21 11:26:15,Click,890270,158534,http://www.facebook.com/UNREFUGEES?utm_medium=...
...,...,...,...,...,...,...
6931,0031K00003AMVE4QAP,2022-08-30 11:03:31,Click,957560,158538,http://cloud.e.unrefugees.org/u4u-unsubscribe?...
6932,0031K00003C0NHpQAN,2022-08-30 12:34:14,Click,957561,1684023,https://www.unrefugees.org/news/five-things-yo...
6933,0031K0000386naMQAQ,2022-02-03 19:50:06,Click,779335,365099,https://www.unrefugees.org/news/army-veteran-a...
6934,0031K0000386naMQAQ,2022-02-03 19:55:46,Click,779335,337899,https://www.unrefugees.org/news/not-even-monso...


In [193]:
data_combined = [df_open, df_sent, df_bounce, df_click, df_donated]
result = pd.concat(data_combined)
result

,contactid,eventdate,eventtype,sendid,bouncecategory,bouncereason,urlid,url,donatemount
0,0031K0000386BslQAE,2022-04-19 07:37:00,Open,883709,NaN,NaN,NaN,NaN,NaN
1,0031K0000386BslQAE,2022-04-19 07:37:00,Open,877618,NaN,NaN,NaN,NaN,NaN
2,0031K00003AMz7nQAD,2022-04-19 07:41:00,Open,886508,NaN,NaN,NaN,NaN,NaN
3,0031K00003AO0VcQAL,2022-04-19 08:04:00,Open,889440,NaN,NaN,NaN,NaN,NaN
4,0031K00003877mQQAQ,2022-04-19 08:05:00,Open,889440,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
24460,0031K00003BbhQEQAZ,NaT,Donate,927013,NaN,NaN,NaN,NaN,NaN
24461,0031K00003Bbjz7QAB,NaT,Donate,924164,NaN,NaN,NaN,NaN,NaN
24484,0031K00003C2dFnQAJ,2022-06-22 00:00:00,Donate,957534,NaN,NaN,NaN,NaN,20.6
24485,0031K00003C2dJpQAJ,2022-06-24 00:00:00,Donate,927012,NaN,NaN,NaN,NaN,30.0


In [194]:
result.to_csv('data_combined.csv')
files.download('data_combined.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>